In [14]:
import pandas as pd
import xgboost as xgb
import category_encoders as ce
import numpy as np
import glob
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import time
import datetime

In [15]:
"""
1.与えられたデータを使えるように読み込む
"""
#　sample_submissionの読み込み
sample_submission = pd.read_csv(r"data/sample_submission.csv", dtype = {'ID':'object', '取引価格（総額）_log':'float'})

# test.csvの読み込み
test_df = pd.read_csv('data/test.csv')

# train.csvの読み込み
train_files = glob.glob('data/train/*.csv') # 条件に当てはまるすべてのパスを抜き取り、リストに格納する
train_df_list = [] # 空のリストを作成し、読み込んだdfを格納していく(最終的に結合するため)
for file in train_files: #　それぞれのファイルに対して
    df = pd.read_csv(file) # ファイルを読み込み
    train_df_list.append(df) #　リストに格納
train_df = pd.concat(train_df_list) #　一つのまとまったデータとしてほしいので結合する。

/Users/masato/miniforge3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [16]:
'''
2.データがどのようなものなのか確認する
'''
# 各カラムに含まれている値を表示するための関数
def check_df_value(df):
    print(df.columns)
    print('======================================\n')
    for col in df.columns:
        print('{}:\n{}'.format(col, df[col].value_counts().keys()))
        print('======================================\n')
check_df_value(train_df)

Index(['ID', '種類', '地域', '市区町村コード', '都道府県名', '市区町村名', '地区名', '最寄駅：名称',
       '最寄駅：距離（分）', '間取り', '面積（㎡）', '土地の形状', '間口', '延床面積（㎡）', '建築年', '建物の構造',
       '用途', '今後の利用目的', '前面道路：方位', '前面道路：種類', '前面道路：幅員（ｍ）', '都市計画', '建ぺい率（％）',
       '容積率（％）', '取引時点', '改装', '取引の事情等', '取引価格（総額）_log'],
      dtype='object')

ID:
Int64Index([44040192, 27022559, 12161066,  7033332, 28161251, 25003237,
            28155110, 12036305, 25027817, 27131114,
            ...
            13297654, 14014480, 13299703, 29001722,  8044541, 29009918,
            15038468, 13444115, 13194462, 22032378],
           dtype='int64', length=677392)

種類:
Index(['中古マンション等'], dtype='object')

地域:
Float64Index([], dtype='float64')

市区町村コード:
Int64Index([13111, 13108, 13112, 13103, 13104, 13109, 13119, 13102, 13120,
            13115,
            ...
             8564, 40604, 21302, 40341, 15108, 40213, 34309, 40212, 40345,
            27147],
           dtype='int64', length=619)

都道府県名:
Index(['東京都', '神奈川県', '大阪府', '兵庫県', '埼玉県

In [17]:
"""
3.データの前処理

訓練用データだけでなくテスト用データも同じように処理する必要があるためtrainとtestは結合して整形する。
"""
train_df['test'] = 0 # test用データでないので0
test_df['test'] = 1 # test用データなので1
df = pd.concat([train_df, test_df]) #　結合

# データ前処理の関数。dfを入れたら前処理を経たdfが出てくる
def data_pre(df):
    nonnull_list = [] #　空のリスト作成
    for col in df.columns: # dfのカラムの数だけ繰り返し
        nonnull = df[col].count() #nonnull = あるカラムの個数をカウントしている
        if nonnull == 0: # もしdf[col].count() == 0なら
            nonnull_list.append(col) # nonnull_listにそのカラムを追加する
    df = df.drop(nonnull_list, axis=1) # nonnull_listに含まれているすべてのカラムを削除する（必要ないため）

    df = df.drop("市区町村名", axis=1) # 評価にあまり影響しないと考え削除(ホントは影響するが、うまく処理するのが難しい)

    df = df.drop("種類", axis=1) # 評価にあまり影響しないと考え削除(ホントは影響するが、うまく処理するのが難しい)

    dis = {
        "30分?60分":45,
        "1H?1H30":75,
        "2H?":120,
        "1H30?2H":105
    } # 文字情報より数値データのほうが分析しやすいためこのような対応の辞書を作成
    df["最寄駅：距離（分）"] = df["最寄駅：距離（分）"].replace(dis).astype(float) #辞書形式の対応表に基づいて置換

    df["面積（㎡）"] = df["面積（㎡）"].replace("2000㎡以上", 2000).astype(float) # "2000以上"を一律2000に置換


    built_list = {} # 建築年数を数値で表示するための置換リストの準備
    for i in df["建築年"].value_counts().keys():
        if "平成" in i:
            num = float(i.split("平成")[-1].split("年")[0])
            year = 33 - num
        if "令和" in i:
            num = float(i.split("令和")[-1].split("年")[0])
            year = 3 - num
        if "昭和" in i:
            num = float(i.split("昭和")[-1].split("年")[0])
            year = 96 - num
        built_list[i] = year
    built_list["戦前"] = 76
    df["建築年"] = df["建築年"].replace(built_list) # 作成した辞書をもとに置換

    year = {
        "年第１四半期": ".25",
        "年第２四半期": ".50",
        "年第３四半期": ".75",
        "年第４四半期": ".99"
    }
    year_list = {}
    for i in df["取引時点"].value_counts().keys():
        for k, j in year.items():
            if k in i:
                year_rep = i.replace(k, j)
        year_list[i] = year_rep
    df["取引時点"] = df["取引時点"].replace(year_list).astype(float)
    
    cols = ["都道府県名", "地区名", "最寄駅：名称", "間取り", "建物の構造", "用途", "今後の利用目的", "都市計画", "改装", "取引の事情等"]
    ce_df = ce.OrdinalEncoder(cols=cols, handle_unknown='impute') # カテゴリーエンコーダーで数値でないものを分析しやすいように変換
    df = ce_df.fit_transform(df) 
    
    return df
    
df = data_pre(df) # 前処理の実行
train_df = df[df['test'] == 0] #train, testの分割
test_df = df[df['test'] == 1]

train_df = train_df.drop("test", axis = 1) #  不必要なカラムの削除
test_df = test_df.drop(["test", "取引価格（総額）_log"], axis = 1)

In [18]:
'''
4.データがどのようなものなのか確認する
'''
check_df_value(train_df)
check_df_value(test_df)

Index(['ID', '市区町村コード', '都道府県名', '地区名', '最寄駅：名称', '最寄駅：距離（分）', '間取り', '面積（㎡）',
       '建築年', '建物の構造', '用途', '今後の利用目的', '都市計画', '建ぺい率（％）', '容積率（％）', '取引時点',
       '改装', '取引の事情等', '取引価格（総額）_log'],
      dtype='object')

ID:
Int64Index([44040192, 27022559, 12161066,  7033332, 28161251, 25003237,
            28155110, 12036305, 25027817, 27131114,
            ...
            13297654, 14014480, 13299703, 29001722,  8044541, 29009918,
            15038468, 13444115, 13194462, 22032378],
           dtype='int64', length=677392)

市区町村コード:
Int64Index([13111, 13108, 13112, 13103, 13104, 13109, 13119, 13102, 13120,
            13115,
            ...
             8564, 40604, 21302, 40341, 15108, 40213, 34309, 40212, 40345,
            27147],
           dtype='int64', length=619)

都道府県名:
Int64Index([44, 36, 27, 37, 42, 47, 10,  1, 34, 26, 38, 11, 15, 33, 35, 19, 13,
             8, 28,  3, 40, 18, 16,  6, 45,  9,  5,  4, 24, 39, 32, 17, 31, 14,
             7, 29, 23, 43, 30, 12,  2, 20, 21, 46

In [26]:
import time
from contextlib import contextmanager
from sklearn.preprocessing import StandardScaler
@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f'[{name}] done in {time.time() - t0:.0f} s')

In [29]:
def first_dataset(train_df, test_df, col = "取引価格（総額）_log"):
    X_train = train_df.drop([col], axis=1)
    y_train = train_df[col]
    X_test = test_df
    
    std = StandardScaler()
    std.fit(X_train)
    X_train = std.transform(X_train).astype(np.float32)
    X_test = std.transform(X_test).astype(np.float32)

    return {'X_train': X_train, 'X_test': X_test, 'y_train': y_train}

In [30]:
df = first_dataset(train_df, test_df)

In [34]:
import lightgbm as lgbm
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(df['X_train'], df['y_train'], test_size=0.2, random_state=0)

train_dataset = lgbm.Dataset(data=X_train, label=y_train, free_raw_data=False)
test_dataset = lgbm.Dataset(data=X_valid, label=y_valid, free_raw_data=False)
final_train_dataset = lgbm.Dataset(data=df['X_train'], label=df['y_train'], free_raw_data=False)

lgbm_params = {
    'objective': 'regression',
}

evaluation_results = {}
clf = lgbm.train(train_set=train_dataset,
                 params=lgbm_params,
                 valid_sets=[train_dataset, test_dataset], 
                 valid_names=['Train', 'Test'],
                 evals_result=evaluation_results,
                 num_boost_round=500,
                 early_stopping_rounds=100,
                 verbose_eval=20
                )
                
clf_final = lgbm.train(train_set=final_train_dataset,
                      params=lgbm_params,
                      num_boost_round=500,
                      verbose_eval=0
                      )

y_pred = clf_final.predict(df['X_test'])


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009513 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1435
[LightGBM] [Info] Number of data points in the train set: 541913, number of used features: 18
[LightGBM] [Info] Start training from score 7.222474
Training until validation scores don't improve for 100 rounds
[20]	Train's l2: 0.0365966	Test's l2: 0.0364197
[40]	Train's l2: 0.0287394	Test's l2: 0.0285785
[60]	Train's l2: 0.0259546	Test's l2: 0.0258358
[80]	Train's l2: 0.0244504	Test's l2: 0.0243863
[100]	Train's l2: 0.0233585	Test's l2: 0.0233335
[120]	Train's l2: 0.022544	Test's l2: 0.0225564
[140]	Train's l2: 0.0219975	Test's l2: 0.02205
[160]	Train's l2: 0.0215137	Test's l2: 0.0216013
[180]	Train's l2: 0.0211052	Test's l2: 0.021236
[200]	Train's l2: 0.0208091	Test's l2: 0.0209806
[220]	Train's l2: 0.020539	Test's l2: 0.020747

In [35]:
y_pred

array([7.07499699, 7.24146217, 7.45733117, ..., 7.48909611, 7.34558834,
       7.35699819])

In [41]:
from heamy.dataset import Dataset
from heamy.estimator import Regressor, Classifier
from heamy.pipeline import ModelsPipeline

from rgf.sklearn import RGFClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

In [42]:
ds = Dataset(preprocessor=first_dataset, use_cache=False)

In [43]:
et_params = {'n_estimators': 100, 'max_features': 0.5, 'max_depth': 18, 'min_samples_leaf': 4, 'n_jobs': -1}
rf_params = {'n_estimators': 125, 'max_features': 0.2, 'max_depth': 25, 'min_samples_leaf': 4, 'n_jobs': -1}
rgf_params = {'algorithm': 'RGF_Sib', 'loss': 'Log'}

In [44]:
from keras.layers import Dense
from keras.models import Sequential

def NuralNetClassifier(X_train, y_train, X_test, y_test=None):
    input_dim = X_train.shape[1]
    
    model = Sequential()
    model.add(Dense(12, input_dim=input_dim, activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=30, batch_size=10, verbose=0)
    y_pred = numpy.ravel(model.predict(X_test))
    
    return y_pred

ModuleNotFoundError: No module named 'keras'